In [1]:
import numpy as np
import pandas as pd
import sqlite3 

## Data preprocessing
Contains methods to:
1. Get the csv student attendance.
2. Extract session information from columns.
3. Clean the original data i.e., remove NaNs, correct data types.

In [2]:
class PreprocessModuleRegister:
    def __init__(self, folder_name, file_name):
        self.folder = folder_name
        self.file = file_name 

    def get_session_details(clm):
        '''
        Extract details of sessions from the columnn names
        Input:
            clm: the columnn names, list type 
        Return: 
            df_details: the session details, dataframe type
        '''
        # separate details
        df_details = pd.DataFrame(clm, columns= ["details"])
        df_details = df_details["details"].str.split("\n", expand= True)  
        df_details.columns = [
            "semester_week", "date", "time",
            "lecture_type", "room_id", "EMPTY"
        ] 

        # further separate columns by deliminators
        old_clms = ["semester_week", "date", "time"]
        new_clms = [
            ["semester","week"], 
            ["day", "date"], 
            ["start_time", "end_time"]
        ]
        delims = [".", " ", "-"]

        for c in range(len(old_clms)):
            df_details[new_clms[c]] = df_details[old_clms[c]].str.split(
                delims[c], 
                expand= True
            )

        new_clms = ["date", "date", "semester", "week", "room_id"]
        delims = ["(", ")", "S", "W", "..."]
        for c in range(len(new_clms)):
            df_details[new_clms[c]] = df_details[new_clms[c]].str.strip(delims[c])
        
        # format to datetime columns SQL standard:
        # https://www.sqlite.org/lang_datefunc.html
        time_clm = ["start_time", "end_time"]
        for t in time_clm:
            day = df_details["date"].str[:2]
            month = df_details["date"].str[3:5]
            year = df_details["date"].str[6:10]
            time = df_details[t].str.strip() + ":00"

            df_details[t] = year + "-" + month + "-" + day + " " + time
            
        # add session number column (used to join later)
        idx = 0
        df_details.insert(loc= idx, column= "session", value= df_details.index+1)

        # remove redundant columns 
        df_details.drop(["semester_week", "time", "EMPTY"],
            axis = 1, 
            inplace = True
        )

        return df_details

    def get_module_attendance_csv(self):
        '''
        Reads .csv and returns the sessions 
        and module attendance per session.
        Input:
            Module attendance details from csv files
        Return:
            df_module_attendance
            df_sessions
        '''
        df_module_attendance = pd.read_csv(
            "./"+ self.folder+ "/"+self.file+".csv", 
            index_col= 0
        )
        clm = df_module_attendance.columns.to_list()
        df_sessions = PreprocessModuleRegister.get_session_details(clm)

        df_module_attendance.columns = np.arange(
            len(df_module_attendance.columns)
        )
        return (df_sessions, df_module_attendance)

    def clean_module_attendance(self, df_module_atten):
        '''
        Cleans module attendance data by:
            + Replace the original strings with boolean or None
            + Remove NaN rows and columns 
        Input: 
            df_module_atten
        Return: 
            df_module_atten (cleaned)
        '''
        df_module_atten.index.names = ["sid"] # renames index (module id = "sid")
        df_module_atten.columns += 1

        old_vals = ['Ex', 'GPS', 'X']
        new_vals = [None, True, False]
            
        for v in range(len(old_vals)):
            df_module_atten.replace(old_vals[v], new_vals[v], inplace = True)

        df_module_atten.dropna(
            axis = 0, 
            how = 'all',
            inplace = True
        ) # drops rows with NaN 
        df_module_atten.dropna(
            axis = 1,
            how = 'all', 
            inplace = True
        ) # drops columns with NULL
        return df_module_atten

In [3]:
# files =  ["22COA111ModuleRegister", "22COA122ModuleRegister"]
# for file in files:
#     # print(get_module_attendance_csv(folder, file))

## Database operations 
Contains methods to:
1. Connect to database (and check it it has been connected).
2. Write dataframe to existing/newly created database.
3. Replace table if it already exists in the database with the latest version.

In [4]:
'''
*I found changing the type for datatypes other than 'INTEGER' in python does 
not guarentee the type is maintainted in SQLite with 'to_sql' alone.
It is instead stored as a storage class rather than a specific datatype:
https://www.sqlite.org/datatype3.html
Datatypes can be concretely changed using this function with sqlalchemy:
https://www.sqlalchemy.org/
'''
class ModuleRegisterDB:
    def __init__(self, database_name, table_name, df_module_data):
        self.db = database_name
        self.table = table_name
        self.df = df_module_data

    def connect_to_db(self):
        '''
        Connects to an SQlite database.
        If the database does not exist it will be created.
        Input: 
            database_name (self.db)
        Return:
            conn
        '''
        try: # try connecting to the database
            conn = sqlite3.connect(self.db)
            return conn
        except Exception as Error:
            print(Error) # if it fails return the error
            if conn is not None:
                conn.close()
        
    def write_df_to_db(self):
        '''
        Write new dataframe to database if it does not already exist.
        Input:
            table_name (self.table)
            df_module_data (self.df)
        Return:
            None (Dataframe is written into database.)
        '''
        conn = ModuleRegisterDB.connect_to_db(self)
        if conn is not None:
            cur = conn.cursor()
            try:
                # create table with correct datatypes*
                if "Sessions" in self.df.columns:
                    cur.execute (
                        "CREATE TABLE " + self.table + ''' (
                        session INTEGER,
                        date DATE,
                        lecture_type VARCHAR, 
                        room_id VARCHAR, 
                        semester INTEGER, 
                        week INTEGER, 
                        day VARCHAR, 
                        start_time DATETIME, 
                        end_time DATETIME);
                        '''
                    )
                    self.df.to_sql(
                        name= self.table,
                        con= conn,
                        if_exists= "replace",
                        index= False
                    )
                else:
                    self.df.to_sql(
                        name= self.table,
                        con= conn,
                        if_exists= "replace",
                        index= False
                    )
            except sqlite3.OperationalError:
                ModuleRegisterDB.replace_sql_tbl(self)
            print("Dataframe has been successfully written to SQL!")
        else:
            print("Connection to database failed. Please try again.")

    def replace_sql_tbl(self):
        '''
        If the SQL table already exists 
        the table is replaced with its latest version.
        Input:
            table_name (self.table)
            df_module_data (self.df)
        Return:
            None (Existing sql table is replaced.)
        '''
        conn = ModuleRegisterDB.connect_to_db(self)
        if conn is not None:
            cur = conn.cursor()
            cur.execute("PRAGMA foreign_keys = off;")
            cur.execute("BEGIN TRANSACTION;")
            cur.execute("ALTER TABLE " + self.table + ''' RENAME
                TO ''' + self.table + "_old;")
            cur.execute(
                "CREATE TABLE " + self.table + ''' (
                session INTEGER,
                date DATE,
                lecture_type VARCHAR, 
                room_id VARCHAR, 
                semester INTEGER, 
                week INTEGER, 
                day VARCHAR, 
                start_time DATETIME, 
                end_time DATETIME); 
                '''
            )
            # writes new data to the table created
            self.df.to_sql(
                name= self.table,
                con= conn,
                if_exists= "append",
                index= False
            )
            cur.execute("DROP TABLE " + self.table + "_old;")
            cur.execute("PRAGMA foreign_keys = on;")
            conn.commit()
            conn.close()
        else:
            print("Connection to database failed. Please try again.")

    def get_module_details_from_db(self):
        conn = ModuleRegisterDB.connect_to_db(self)
        if conn is not None:
            cur = conn.cursor()
            if "Sessions" in self.table:
                df_mod_sessions = pd.read_sql("SELECT * FROM " + self.table, 
                    con= conn
                )
                df_mod_atten = pd.read_sql("SELECT * FROM " + self.table.replace(
                    "Sessions", ""), 
                    con= conn
                )
            else:
                df_mod_sessions = pd.read_sql("SELECT * FROM " + self.table + '''
                    Sessions''', 
                    con= conn,
                    index_col= ["sid"]
                )
                df_mod_atten = pd.read_sql("SELECT * FROM " + self.table, 
                    con= conn
                )
            return df_mod_sessions, df_mod_atten
        else:
            print("Connection to database failed. Please try again.")

#==============================================================================

# Testing

In [5]:
folder = "cop504cwdata"
file = "22COA111ModuleRegister"

pp = PreprocessModuleRegister(folder, file)
df_sessions, df_mod_atten = pp.get_module_attendance_csv()
df_clean_mod_atten = pp.clean_module_attendance(df_mod_atten)
df_clean_mod_atten
df_sessions

database = "CWDatabase.db"
table = "COA111Sessions" 
modb = ModuleRegisterDB(
    database_name= database, 
    table_name= table, 
    df_module_data= df_sessions
)
modb.connect_to_db()
modb.write_df_to_db() # it also replaces table if it already exists
mod_sessions, mod_atten = modb.get_module_details_from_db()
#==============================================================================


Dataframe has been successfully written to SQL!


Desiging GUI for Sections 1.3 to 1.5.
Get ideas: https://www.youtube.com/watch?v=M43u_K_ZZfQ
User options:
1. Add csv files by BROWSING button (looks for file in computer)
2. Look up student id (a. show weekly attendancre graph, )
3. Look up module code and week number 
4. Add new student attendance info (show updated colour coded files (white if normal, red if disengaged etc.))
5. Add new session info  (")
6. Remove "
7. Remove "
8. Refuse duplicates
Extra: Option to save student record as a pdf 